# Factorio 品质优化建模

In [1]:
import numpy as np
import sympy as sp
import pandas as pd
from IPython.display import display, Math, Markdown

np.set_printoptions(precision=6, suppress=True, floatmode='maxprec')
pd.set_option('display.precision', 6)

# 基础设定

## 参数设定

- 一个完整循环包含生产流程和回收流程
- 设一个流程的总产能为$P$(等于1+所有产能加成), 品质加成概率为$Q$; 机器基础产能加成为$P_0$, 插槽位为N, 使用$n_1$个产能插件和$n_2$个品质插件; 又有当前可用的产能科技加成为$P_s$, 产能插件加成为$p$, 品质插件概率为$q$; 则:
$$ \begin{align*}
N & = n_1 + n_2 \\
P & = 1 + P_0 + P_S + n_1 p \\
Q & = n_2 q \\
\end{align*}
$$
- 生产流程中有限制
$$ 1 \le P \le 4, 0 \le Q \le 0.5 $$ 
- 回收流程固定有
$$P = \frac{1}{4}, Q = 4 q$$
- 星岩再处理流程中，若将3种星岩视为一类物品，只考虑总数，则固定有
$$P = 0.8, Q = 2 q$$
- 另：不同工厂还有不同的制造速度倍率V，此参数不影响转化率，但影响生产速率。在基础倍率的基础上，工厂品质每升一级，制造速度增加30%，传说级增加150%。

- 按产能加成、插槽数和速度倍率分，常见工厂类型如下
| 代表工厂 | $P_0$ | N | V |
|------|------|------|------|
| 离心机/破碎机 | 0 | 2 | 1 |
| 化工厂 | 0 | 3 | 1 |
| 组装机3 | 0 | 4 | 1.25 |
| 生物室 | 0.5 | 4 | 2 |
| 铸造厂 | 0.5 | 4 | 4 |
| 电磁工厂 | 0.5 | 5 | 2 |
| 低温工厂 | 0 | 8 | 2 |
| 回收机 | -0.75 | 4 | 0.5(*16) |

## 转换矩阵

- 品质加成概率Q的机制是: 基于当前品质等级, 升1级的概率为$\frac{9}{10}Q$, 升2级的概率为$\frac{9}{10^2}Q$, ..., 升到顶级(传说)的概率为$\frac{1}{10^{n-1}}Q$

- 因此，设参与一轮生产流程的物品，初始时的品质分布为$I = \begin{bmatrix} I_1 & I_2 & I_3 & I_4 & I_5 \end{bmatrix}$, 每个等级的工厂配置分别为$P_i, Q_i$, 结束时的品质分布为 $O = \begin{bmatrix} O_1 & O_2 & O_3 & O_4 & O_5 \end{bmatrix}$, 则使得$O = I T_p$的转换矩阵$T_p$为:

In [2]:
P_1, P_2, P_3, P_4, P_5 = sp.symbols('P_1, P_2, P_3, P_4, P_5')
Q_1, Q_2, Q_3, Q_4, Q_5 = sp.symbols('Q_1, Q_2, Q_3, Q_4, Q_5')
T = sp.Matrix([
    P_1 * sp.Matrix(1, 5, [ 1-Q_1 , 0.9*Q_1 , 0.09*Q_1 , 0.009*Q_1 , 0.001*Q_1 ]),
    P_2 * sp.Matrix(1, 5, [ 0 , 1-Q_2 , 0.9*Q_2 , 0.09*Q_2 , 0.01*Q_2 ]),
    P_3 * sp.Matrix(1, 5, [ 0 , 0 , 1-Q_3 , 0.9*Q_3 , 0.1*Q_3 ]),
    P_4 * sp.Matrix(1, 5, [ 0 , 0 , 0 , 1-Q_4 , Q_4 ]),
    P_5 * sp.Matrix(1, 5, [ 0 , 0 , 0 , 0 , 1])
])
display(Math('T = ' + sp.latex(T)))

<IPython.core.display.Math object>

- 特别地, 当除传说级外所有机器的配置均相同, 参数为$P, Q$, 传说级$P_5 = P_L$时, 可简化为

In [3]:
P, P_L, Q = sp.symbols('P, P_L, Q')
T_p = T.evalf(subs={P_1:P, P_2:P, P_3:P, P_4:P, Q_1:Q, Q_2:Q, Q_3:Q, Q_4:Q}).subs(P_5, P_L)
display(Math(r"T_p = " + sp.latex(T_p)))

<IPython.core.display.Math object>

- 对用全传说品质插件的回收机，其$P = 0.25, Q = 0.248 \approx 0.25 $，传说级 $P_L = \begin{cases} 0.25 & if 回收 \\ 1 & if 不回收 \end{cases}$

In [4]:
T_r = T_p.evalf(subs={Q:0.25, P:0.25})
display(Math(rf'''T_r = {sp.latex(T_r)}'''))

<IPython.core.display.Math object>

- 对全传说品质插件的破碎机(星岩再处理)，$P=0.8, Q=0.125,P_L=1$

In [5]:
T_b = T_p.evalf(subs={Q:0.125, P:0.8, P_L:1})
display(Math(rf'''T_b = {sp.latex(T_b)}'''))

<IPython.core.display.Math object>

- 考察一组完整的生产-回收循环，初始时原料-产品为$\begin{bmatrix} I & O \end{bmatrix}_i$，结束后变为$\begin{bmatrix} I & O \end{bmatrix}_{i+1}$，其转换矩阵$T$:
$$
\begin{bmatrix} I & O \end{bmatrix}_{i+1} =
\begin{bmatrix} I & O \end{bmatrix}_i T =
\begin{bmatrix} I & O \end{bmatrix}_i \begin{bmatrix}
A_1 & T_p \\
T_r & A_2 \end{bmatrix}
$$
- 或者写成
$$
\begin{align*}
I_{i+1} & = I_i A_1 + O_i T_r \\
O_{i+1} & = O_i A_2 + I_i T_p
\end{align*}
$$

- 如果原料/产物的全部用于合成/分解，则对应的$A_i=0$; 如果传说级原料/产物将全部保留，则对应的$A_i=A=\begin{bmatrix}
0 & \cdots & 0 \\
\vdots & \ddots & \vdots \\
0 & \cdots & 1
\end{bmatrix}_{5 \times 5}$

In [6]:
A = sp.zeros(5); A[4,4]=1;
display(Math(r'A = ' + sp.latex(A)))

<IPython.core.display.Math object>

# 几种优化途径

## 星岩再处理
- $O_{i+1} = O_i T_b$

In [7]:
display(Math(rf'T_b = {sp.latex(T_b)}'))

<IPython.core.display.Math object>

## 回收得到自身
- $O_{i+1} = O_i T_r(P_L=1) = O_i T_R$

In [8]:
T_R = T_r.subs(P_L, 1).expand()
display(Math(rf'T_R = {sp.latex(T_R)}'))

<IPython.core.display.Math object>

## 作为产物优化(分-合)
- 作为产物优化时，原料全部合成，而传说级产物不再分解，有$A_1 = 0, A_2 = A, T_p=T_p(P_L=P_M), T_r=T_r(P_L=0)$, 其中$P_M$为该机器的最大产能

  考察过程 $\begin{bmatrix} 0, O_i \end{bmatrix} \overset{分解}{\rightarrow} \begin{bmatrix} I_{i+1}, O_i' \end{bmatrix} \overset{合成}{\rightarrow} \begin{bmatrix} 0, O_{i+1} \end{bmatrix} $, 有

$$ \begin{align*}
O_{i+1} & = O_i' A_2 + I_{i+1} T_p \\
 & = O_i A_2 + O_i T_r T_p \\
 & = O_i (A + T_r(P_L=0) T_p(P_L=P_M)) \\
 & = O_i T_O
\end{align*}$$

In [9]:
P_M = sp.symbols('P_M')
T_O = (A + T_r.subs(P_L, 0) * T_p.subs(P_L, P_M)).expand()
display(Math(rf'T_O = {sp.latex(T_O)}'))

<IPython.core.display.Math object>

## 作为原料优化(合-分)
- 作为原料优化时，产物全部分解一次，但传说级材料不再合成, 有 $A_1 = A, A_2 = 0, T_p = T_p(P_L=0), T_r = T_r(P_L=0.25)$

  考察过程 $\begin{bmatrix} I_i, 0 \end{bmatrix} \overset{合成}{\rightarrow} \begin{bmatrix} I_i', O_{i+1} \end{bmatrix} \overset{分解}{\rightarrow}  \begin{bmatrix} I_{i+1}, 0 \end{bmatrix} $, 有

$$ \begin{align*}
I_{i+1} & = I_i' A_1 + O_{i+1} T_r(P_L=0.25) \\
 & = I_i A_1 + I_i T_p(P_L=0) T_r(P_L=0.25) \\
 & = I_i(A + T_p(P_L=0) T_r(P_L=0.25)) \\
 & = I_i T_I
\end{align*}$$

In [10]:
T_I = (A + T_p.subs(P_L, 0) * T_r.subs(P_L, 0.25)).expand()
display(Math(rf'T_I = {sp.latex(T_I)}'))

<IPython.core.display.Math object>

## 注意到
$$T_O - T_I = \begin{bmatrix}
0_{4 \times 4} & R_{4 \times 1} (P_M - P) \\
0_{1 \times 4} & 0
\end{bmatrix}$$
其中$R_i = T_r[i, 5], i=1 \dots 4$

  可知: 就单步转换而言，分-合式刷品质相比合-分式只在各级别到传说的转化率上有一个提升，且该提升正比于进行传说级生产的工厂上的产能插件加成$\Delta P = P_M - P = n*p$, 以及回收机上各级别到传说的转化率$R_i = 10^{-4+i} P_r Q_r$

In [11]:
display(Markdown(r'''若令$r_i = \frac{R_i(P_M-P)}{T_I[i,5]}$ (表示各级别的提升倍率)，则$r_i = \frac{T_r[i,5]}{T_I[i,5]}(P_M-P) = %s^T (P_M-P)$''' %
    sp.latex(sp.Matrix([1/(i/r) for i, r in zip(T_I[0:4,4],T_r[0:4,4])]).T)))

若令$r_i = \frac{R_i(P_M-P)}{T_I[i,5]}$ (表示各级别的提升倍率)，则$r_i = \frac{T_r[i,5]}{T_I[i,5]}(P_M-P) = \left[\begin{matrix}\frac{1}{30.0 P Q + 1.0 P} & \frac{1}{21.0 P Q + 1.0 P} & \frac{1}{12.0 P Q + 1.0 P} & \frac{1}{3.0 P Q + 1.0 P}\end{matrix}\right]^T (P_M-P)$

# 转化率

## 定义
- 合-分式定义状态为$X_i = I_i$, 其它情况下定义$X_i = O_i$
- 每经过一个完整的生产-回收周期，$X_{i+1} = X_i T + X_i^+$
- 若只在0时刻给予单位输入$X_0 = \begin{bmatrix} 1 & 0 & 0 & 0 & 0 \end{bmatrix}, X_i^+ = 0$, 若$\lim_{n \to \infty}X_n = \begin{bmatrix} \dots & \alpha \end{bmatrix}$存在，则称$\alpha$为（普通级到传说级的）转化率

## 分析
- 以上的条件下显然 $X_n = X_0 T^n$

- 容易验证，在限制条件$1 \le P \le P_M \le 4, 0 \le Q \le 0.5$下，每种$T$都满足
$$T = \begin{bmatrix}
 a_{11} & a_{12} & a_{13} & a_{14} & a_{15} \\
 0 & a_{22} & a_{23} & a_{24} & a_{25} \\
 0 & 0 & a_{33} & a_{34} & a_{35} \\
 0 & 0 & 0 & a_{44} & a_{45} \\
 0 & 0 & 0 & 0 & 1 \\
\end{bmatrix} $$
$$ 0 < a_{ij} < 1, 1 \le i,j \le 5 $$
$$ \sum_{j=1}^5 a_{ij} \le 1, 1 \le i \le 4 $$
  用矩阵/向量范数的方式表示为
$$\left\|T\right\|_\infty = \max_{i=1\dots5} \left\|T_i\right\|_1 = 1$$
其中对$i=1\dots4$, 当且仅当$P=P_M=4$时取到1。物理意义是优化后的产品总数不会多于之前的总数

- 由$T$是末位为1的上三角矩阵，易知$T^n$都是末位为1的上三角矩阵

- **引理**：设 $X = (x_1, x_2, x_3, x_4, x_5)$ 满足 $P(X): \begin{cases} x_i \ge 0 \\ \left\|X\right\|_1 \le 1 \end{cases}$ , $Y = (y_1, y_2, y_3, y_4, y_5) = X T$, 若$T$满足 $\begin{cases} 0 \le T_{ij} \le 1 \\ 0 \le \sum_{j} T_{ij} \le 1 \end{cases}$,  易知$Y$也满足: 
$$ \begin{cases}
y_j = \sum x_i T_{ij} \ge 0
\\
\left\|Y\right\|_1 = \sum_{j} \sum_{i} x_i T_{ij} = \sum_{i} (x_i \sum_{j} T_{ij}) \le \sum_{i} x_i = \left\|X\right\|_1 \le 1
\end{cases} $$
  即 $P(X) => P(XT)$
- 对$T^n$的每一行$T_i^n$，有 $T_i^{n+1} = T_i^{n} T$，其中$T$满足 $\begin{cases} 0 \le T_{ij} \le 1 \\ 0 \le \sum_{j} T_{ij} \le 1 \end{cases}$. 故$P(T_i^1)$且$P(T_i^{n}) => P(T_i^{n+1})$, 对$n \in N$使用数学归纳法得$P(T_i^n), n \in N$. 即$T^n$的每个元素都$\ge 0$，且每个行向量的$L1$范数都$\le 1$

- 故$T^n$也满足$T$的上述3个特征

- 将$T$写为分块矩阵$T = \begin{bmatrix} K_{4 \times 4} & b_{4 \times 1} \\ 0_{1 \times 4} & 1_{1 \times 1}\end{bmatrix}$, 则有
$$\begin{align*}
T^1 &= \begin{bmatrix} K & b \\ 0 & 1\end{bmatrix} \\
T^2 &= \begin{bmatrix} K^2 & K b + b \\ 0 & 1\end{bmatrix} \\
T^3 &= \begin{bmatrix} K^3 & K^2 b + K b + b \\ 0 & 1\end{bmatrix} \\
& \dots \\
T^n &= \begin{bmatrix} K^n & b_n \\ 0 & 1\end{bmatrix}, 其中b_n = K^{n-1}b + b_{n-1} = \sum_{i=1}^{n-1} K^i b = \frac{I-K^n}{I-K}b
\end{align*}$$
- 由于
$$\left\|K\right\|_\infty = \max_{i=1\dots4} \left\|K_i\right\|_1 < \max_{i=1\dots4} \left\|T_i\right\|_1 <= 1$$
  故
$$\lim_{n \to \infty} K^n = 0, \lim_{n \to \infty} b_n = (I-K)^{-1}b $$
- 即
$$\lim_{n \to \infty} T^n = \begin{bmatrix} 0 & (I-K)^{-1}b \\ 0 & 1\end{bmatrix}$$

## 结论
- 由前，将$T$按前4列、前4行分块$T = \begin{bmatrix} K_{4 \times 4} & b_{4 \times 1} \\ 0_{1 \times 4} & 1_{1 \times 1}\end{bmatrix}$，有
$$ \lim_{n \to \infty} X_n = X_0 \lim_{n \to \infty} T^n =
 X_0 \begin{bmatrix} 0 & (I-K)^{-1}b \\ 0 & 1\end{bmatrix} = 
 X_0 \begin{bmatrix}
 0 & 0 & 0 & 0 & \alpha_1 \\
 0 & 0 & 0 & 0 & \alpha_2 \\
 0 & 0 & 0 & 0 & \alpha_3 \\
 0 & 0 & 0 & 0 & \alpha_4 \\
 0 & 0 & 0 & 0 & 1 \end{bmatrix} =
 \begin{bmatrix} 0 & 0 & 0 & 0 & X_0 \cdot (\alpha_1, \alpha_2, \alpha_3, \alpha_4, 1) \end{bmatrix}
$$
- 对初始1单位的普通级输入$X_0 = \begin{bmatrix} 1 & 0 & 0 & 0 & 0 \end{bmatrix}$，$X_n = \begin{bmatrix} 0 & 0 & 0 & 0 & \alpha_1 \end{bmatrix}$ 即最终会产出$\alpha_1$数量的传说级物品
- 故(普通级到传说级的)转化率 $\alpha = \alpha_1$, 此即为向量 $(I-K)^{-1} b$ 的第一个元素

## 具体计算

In [50]:
Ps, Qs = np.linspace(1, 4, 13), np.linspace(0, 0.5, 9)
indices = pd.MultiIndex.from_product([Ps, Qs], names=['P', 'Q'])
invs = [(sp.eye(4) - T_I[:4,:4].evalf(subs={'P':P,'Q':Q})).inv() for P, Q in indices]

In [52]:
convrates = lambda T: (sp.eye(4) - T[:4,:4]).inv() * T[:4,4]
convrate = lambda T: convrates(T)[0]

In [57]:
data = [(invs[i] * T_I[:4,4].evalf(subs={'P':P,'Q':Q}))[0] for i, (P, Q) in enumerate(indices)]
df = pd.DataFrame(np.array(data, dtype=np.float64), index=indices, columns=['α'])

In [61]:
df0 = df.reset_index().pivot(index='P', columns='Q', values='α')

In [62]:
display(Markdown(f'''
### 星岩再处理
{convrate(T_b)}
### 回收自身
{convrate(T_R)}
### 合-分式优化在不同$P,Q$下的转化率
'''))
display(df0)


### 星岩再处理
0.0213333333333334
### 回收自身
0.000372816077868422
### 合-分式优化在不同$P,Q$下的转化率


Q,0.0000,0.0625,0.1250,0.1875,0.2500,0.3125,0.3750,0.4375,0.5000
P,,,,,,,,,
1.00,0.000373,0.000922,0.001594,0.002385,0.003293,0.004313,0.005442,0.006677,0.008014
1.25,0.000720,0.001701,0.002906,0.004327,0.005952,0.007772,0.009777,0.011955,0.014297
1.50,0.001340,0.003014,0.005066,0.007468,0.010196,0.013224,0.016528,0.020085,0.023873
1.75,0.002439,0.005216,0.008582,0.012476,0.016842,0.021628,0.026784,0.032265,0.038031
2.00,0.004390,0.008909,0.014286,0.020397,0.027133,0.034395,0.042094,0.050155,0.058509
2.25,0.007890,0.015142,0.023555,0.032897,0.042971,0.053610,0.064676,0.076052,0.087642
2.50,0.014268,0.025791,0.038724,0.052665,0.067300,0.082381,0.097715,0.113150,0.128571
2.75,0.026168,0.044311,0.063835,0.084126,0.104745,0.125383,0.145819,0.165902,0.185528
3.00,0.049110,0.077314,0.106106,0.134717,0.162682,0.189727,0.215705,0.240545,0.264227


### 分-合式优化
  由之前的讨论可知，分-合式与合-分式刷品质的$K$完全一致，只在$b$上存在一个倍率差别。因此前者的总转化率也恰好比后者高$r_i = \frac{np}{(1+30Q)P}$倍

### 对数据的讨论
- 回收自身相当于$P = 1, Q = 0$时的合-分式优化，因此必然是转化率最低的方式
- 星岩再处理的转化率，大约相当于产能P=1+80%~90%，使用4个传说质量插件(Q=25%)时的情况
- 分-合式优化的总转化率固定比合-分式高$r_i = \frac{np}{(1+30Q)P}$倍，其中$n$是合成传说级的机器上可增加的产能插件数. P_M = P时两种方式转化率相等.
- 对分-合式优化而言，基础P越大，或者Q越大，则以传说产能插件(P+25%)替代传说品质插件(Q+6.25%)越能获得较大的的转化率提升（P,Q过小时的替换也可能造成转化率降低）。
- 但对合-分式优化，由于基础P和Q增大使得$r_i$的分子减小（可增加的n减小了）而分母变大，可以预期总转化率提升比例小于分-合式优化。

# 生产速率

## 定义
- 若在所有时刻有恒定1单位普通级的补充输入$X_i^+ = X^+ = \begin{bmatrix} 1 & 0 & 0 & 0 & 0 \end{bmatrix}$，由于系统是线性时不变的，故简单地有
$$X_n = X_0^+ T^n + X_1^+ T^{n-1} + X_2^+ T^{n-2} ... + X_{n-1}^+ T$$
  而对每一项$\lim_{n \to \infty} X_i^+ T^{n-i} = \begin{bmatrix} 0 & 0 & 0 & 0 & \alpha \end{bmatrix}$，故
$$\lim_{n \to \infty}\frac{X_n}{n} = \begin{bmatrix} 0 & 0 & 0 & 0 & \alpha \end{bmatrix}$$
  即每周期的产出就是$\alpha$

- 因此，**定义**单位(传奇)产出所需的生产时间为$t = \tau / \alpha$, 生产速率为$v = 1/t = \alpha / \tau$, $\tau$为单位输入$X^+$下的每周期耗时

- 进一步，由于各阶生产和回收流程是并发的，故 $\tau = \max_{对每个工厂} (单位输入下该工厂要处理的物品)*(单位物品的处理时间)$，周期耗时只取决于总耗时最长的工厂
- 故应求在恒定输入$X_i^+ = \begin{bmatrix} 1 & 0 & 0 & 0 & 0 \end{bmatrix}$下，传奇产物达到增长率达到恒定时，各阶工厂需处理的物品数

## 单位输入下的处理量

- 此时，若每周期从系统中带走$\alpha$数量的传奇产物，系统状态应保持恒定：
$$\begin{cases}
X_i^+ & = X^+ = \begin{bmatrix} 1 & 0 & 0 & 0 & -\alpha \end{bmatrix} \\
X_{i+1} & = X_i T + X_i^+ \\
\tilde{X} &= \lim_{i \to \infty} X_i = \lim_{i \to \infty} X_{i+1}
\end{cases}$$
  故有
$$ \tilde{X}(I - T) = X^+ $$

- (因$I - T$不可逆) 仍然将$T$按前4|第5行/列分块：
$$ \begin{bmatrix} \tilde{X}_{1\dots4} & x_5 \end{bmatrix} \begin{bmatrix} I - K_{4\times4} & b_{4\times1} \\ 0 & 1 - 1 \end{bmatrix} = \begin{bmatrix} 1 & 0 & 0 & 0 & \alpha \end{bmatrix} $$

- $T$的上三角性决定了前4项的计算不包含第5项参与：
$$\begin{align*}
\tilde{X}_{1\dots4} (I - K) & = \begin{bmatrix} 1 & 0 & 0 & 0 \end{bmatrix} \\
\tilde{X}_{1\dots4} & = \begin{bmatrix} 1 & 0 & 0 & 0 \end{bmatrix} (I - K)^{-1}
\end{align*}$$

**结论**：若每周期恒定给予1单位普通级原料，则输出稳定时1~4阶的工厂分别需处理
$\begin{bmatrix}\tilde{x}_1 & \tilde{x}_2 & \tilde{x}_3 & \tilde{x}_4 \end{bmatrix} = \begin{bmatrix} 1 & 0 & 0 & 0 \end{bmatrix} (I - K)^{-1}$
的数量

- 另一方面，同样由线性时不变性 $\tilde{X} = \lim_{n \to \infty} X_n = \lim_{n \to \infty}(X_0^+ T^n + X_1^+ T^{n-1} + X_2^+ T^{n-2} ... + X_{n-1}^+ T) = X^+ \lim_{n \to \infty} \sum_{i=1}^{n} T^i$

  等价于只有初始激励$X_0' = X^+$的系统在此后所有时刻的输出求和 $\lim_{n \to \infty} \sum_{i=1}^{n} X_i' = \lim_{n \to \infty} X_0' \sum_{i=1}^{n} T^i = \tilde{X} $

- 特别地，对$\tilde{X}$ 每一位上的元素，有 $\tilde{X}[j] = \lim_{n \to \infty} \sum_{i=1}^{n} X_i'[j],\ j = 1 \dots 4$

### 试证明 $\tilde{X}[1] = \max(\tilde{X})$
- 但一般情形下此式似乎不一定成立。只能证明在中间工厂配置一样($K$具有右下方向的平移不变性)的情况下具有此特性。

- 此时，$K_{i+1,j+1} = K_{i,j}$, 因此
$$ \begin{cases} X_{i+1}[j] = k_1 X{i}[j] \\ X_{i+1}[j+1] = k_1 X_{i+1}[j+1] + k_2 X{i}[j] \end{cases} $$ 对所有$i,j$都有同样的常数$k_1,k_2$且$k_2 < 1 - k_1$

- 设 $y_i = X_i'[1], z_i = X_i'[2]$, 则初始 $y_0 = 1, z_0 = 0$, 递推地有
$$\begin{align*}
y_n & = k_1 y_{n-1} = k_1^n y_0 = k_1^n \\
z_n & = k_1 z_{n-1} + k_2 y_{n-1} \\
 & = k_1 (k_1 z_{n-2} + k_2 y_{n-2}) + k_2 k_1^{n-1} y_0 \\
 & = k_1^2 (z_{n-3} + k_2 y_{n-3}) + k_1 k_2 k_1^{n-2} y_0 + k_2 k_1^{n-1} y_0 \\
 & \dots \\
 & = k_1^n z_0 + n k_2 k_1^{n-1} y_0 = n k_2 k_1^{n-1} \\
 \\
\sum_{i=0}^{n} y_i &= \frac{1 - k_1^{n+1}}{1 - k_1} \\
\sum_{i=0}^{n} z_i &= k_2 \sum_{i=1}^{n} i k_1^{i-1} 
= k_2 \frac{\sum_{i=1}^{n} k_1^{i-1} - n k_1^n}{1-k_1} 
= k_2 \frac{\frac{1-k_1^n}{1-k_1} - n k_1^n}{1-k_1} \\
& < (1 - k_1) \frac{\frac{1-k_1^n}{1-k_1} - n k_1^n}{1-k_1}
  < \frac{1-k_1^n}{1-k_1} = \sum_{i=0}^{n-1} y_i \\
\\
\therefore \  \lim_{n \to \infty} \sum_{i=1}^{n} z_i & \le \lim_{n \to \infty} \sum_{i=1}^{n-1} y_i \\
 \tilde{X}[2] & \le \tilde{X}[1]
\end{align*}$$

- 同理可证$\tilde{X}[3] \le \tilde{X}[1], \tilde{X}[4] \le \tilde{X}[1]$（将$X_{i+1}[1] \overset{k_2}{\rightarrow} X_{i+1}[2]  \overset{k_2}{\rightarrow} X_{i+1}[3] (\overset{k_2}{\rightarrow} X_{i+1}[4])$ 多次转移的效应等效为一步即可）

## 单位物品的处理时间
- 由于总可以通过增加工厂的方式增加绝对生产速度，因此在此只讨论每个品质等级都只用1个工厂时的情况
- 每个合成配方有各自基础的合成时间，除以对应的原料/产物数，可得单位输入所需合成时间$\tau_0$
- 不同工厂有不同基础速度倍率$k_f$
- 工厂本身的品质每提升一级，基础速度倍率增加30%（传说级加到150%），设这个加成为$k_l$
- 每使用一个品质插件3，减少5%；每使用一个产能插件3，减少15%；但总计最低不会低于20%。因此这个加成为 $k_m = max(1 - 0.05 n_1 - 0.15 n_2, 0.2)$
- 则实际单位输入处理时间为 $\tau = {\tau_0}/{k_f k_l k_m} $, 只考虑插件的影响$\tau = {\tau_f}/{k_m}$

## 具体计算

### 单位输入下所需最大处理量
- 记作${Items}_{max}$
- 注意合-分式中此量为待合成的原料数，其它情况下都是待分解的产物数

In [77]:
miditems = lambda T: (sp.eye(4) - T[0:4,0:4]).inv()[0,:]

In [78]:
# Given: Ps, Qs, indices, invs
data = [np.array(invs[i][0,:], dtype=np.float64).flatten() for i, (P, Q) in enumerate(indices)]
df1 = pd.DataFrame(data, index=indices, columns=[f'X{i}' for i in range(1, 5)])

In [79]:
df11 = df1.reset_index().pivot(index='P', columns='Q', values='X1')

In [80]:
display(Markdown(f'''
### 星岩再处理
${sp.latex(miditems(T_b))}$
### 回收自身
${sp.latex(miditems(T_R))}$
### 合-分/分-合
- 如前所述，所有中间工厂配置都相同时，普通级工厂处理量便是最大，因此只展示普通级的量
'''))
display(df11)


### 星岩再处理
$\left[\begin{matrix}3.33333333333333 & 1.0 & 0.4 & 0.16\end{matrix}\right]$
### 回收自身
$\left[\begin{matrix}1.23076923076923 & 0.085207100591716 & 0.0144196631770596 & 0.0024402506915024\end{matrix}\right]$
### 合-分/分-合
- 如前所述，所有中间工厂配置都相同时，普通级工厂处理量便是最大，因此只展示普通级的量


Q,0.0000,0.0625,0.1250,0.1875,0.2500,0.3125,0.3750,0.4375,0.5000
P,,,,,,,,,
1.00,1.230769,1.213270,1.196262,1.179724,1.163636,1.147982,1.132743,1.117904,1.103448
1.25,1.306122,1.281602,1.257985,1.235223,1.213270,1.192084,1.171625,1.151856,1.132743
1.50,1.391304,1.358090,1.326425,1.296203,1.267327,1.239709,1.213270,1.187935,1.163636
1.75,1.488372,1.444288,1.402740,1.363515,1.326425,1.291299,1.257985,1.226347,1.196262
2.00,1.600000,1.542169,1.488372,1.438202,1.391304,1.347368,1.306122,1.267327,1.230769
2.25,1.729730,1.654281,1.585139,1.521545,1.462857,1.408528,1.358090,1.311140,1.267327
2.50,1.882353,1.783972,1.695364,1.615142,1.542169,1.475504,1.414365,1.358090,1.306122
2.75,2.064516,1.935728,1.822064,1.721008,1.630573,1.549168,1.475504,1.408528,1.347368
3.00,2.285714,2.115702,1.969231,1.841727,1.729730,1.630573,1.542169,1.462857,1.391304


### 单位产出所需最大处理量
- 等于 ${{Items}_{max}} / {\alpha}$
- 此量的意义在于：传奇产物的生产效率$v = \alpha / \tau = \frac{\alpha}{{Items}_{max}} \frac{k_f k_l k_m}{\tau_0}$, 正比于工厂总生产速率和插件效率加成，而反比于此量。

In [76]:
display(Markdown(f'''
### 星岩再处理
${sp.latex(miditems(T_b) / convrate(T_b))}$
### 回收自身
${sp.latex(miditems(T_R) / convrate(T_R))}$
### 合-分/分-合
- 仍然是以普通级计算
'''))
display(df11/df0)


### 星岩再处理
$\left[\begin{matrix}156.25 & 46.875 & 18.75 & 7.5\end{matrix}\right]$
### 回收自身
$\left[\begin{matrix}3301.27723516153 & 228.54996243426 & 38.6776859504132 & 6.54545454545454\end{matrix}\right]$
### 合-分/分-合
- 仍然是以普通级计算


Q,0.0000,0.0625,0.1250,0.1875,0.2500,0.3125,0.3750,0.4375,0.5000
P,,,,,,,,,
1.00,3301.277235,1315.966730,750.600316,494.626573,353.402769,266.168371,208.132029,167.420210,137.696634
1.25,1813.070322,753.598127,432.891674,285.496515,203.837343,153.375080,119.837564,96.348242,79.227881
1.50,1038.250667,450.559487,261.851523,173.567042,124.298071,93.747105,73.406546,59.145390,48.743571
1.75,610.337009,276.887277,163.454427,109.290825,78.755520,59.705148,46.967821,38.008107,31.454941
2.00,364.431487,173.102095,104.185855,70.508937,51.276486,39.173583,31.028439,25.268203,21.035498
2.25,219.228299,109.249663,67.293976,46.251332,34.043009,26.273570,20.998392,17.239997,14.460235
2.50,131.932463,69.170550,43.781160,30.668134,22.914794,17.910717,14.474441,12.002543,10.158750
2.75,78.893698,43.684881,28.543410,20.457615,15.567052,12.355532,10.118758,8.490140,7.262347
3.00,46.543185,27.365044,18.559072,13.671054,10.632590,8.594294,7.149438,6.081439,5.265556
